# Performing gradient on remote tensors

In [3]:
import torch as th

In [4]:
import syft as sy

In [6]:
hook = sy.TorchHook(th)

In [7]:
anum = sy.VirtualWorker(hook,id="anum")

In [91]:
x = th.tensor([1.,2,3,4],requires_grad=True).send(anum)
y = th.tensor([1.],requires_grad=True).send(anum)

In [92]:
x

(Wrapper)>[PointerTensor | me:19504699817 -> anum:97251796553]

In [93]:
y

(Wrapper)>[PointerTensor | me:30193878936 -> anum:78250466618]

In [94]:
z = (x+y).sum()

In [95]:
z.backward()

(Wrapper)>[PointerTensor | me:13839466283 -> anum:86551994779]

In [96]:
x = x.grad
print(x)

(Wrapper)>[PointerTensor | me:19504699817 -> anum:97251796553]::grad


In [97]:
x = x.clone().get()

In [98]:
x

tensor([1., 1., 1., 1.])

# Build a simple linear Model

In [12]:
input = th.tensor([[1.,1],[0,1],[1,0],[0,0]],requires_grad=True).send(anum)
target = th.tensor([[1.],[1],[0],[0]],requires_grad=True).send(anum)
weights = th.tensor([[0.],[0]],requires_grad=True).send(anum)

In [15]:
for i in range(10):
    pred = input.mm(weights)
    loss = ((pred-target)**2).sum()
    loss.backward()
    weights.data.sub_(weights.grad*0.1)
    weights.grad *= 0
    print(loss.get().data)
    

tensor(0.0058)
tensor(0.0037)
tensor(0.0024)
tensor(0.0015)
tensor(0.0010)
tensor(0.0006)
tensor(0.0004)
tensor(0.0003)
tensor(0.0002)
tensor(0.0001)


TensorsNotCollocatedException: You tried to call a method involving two tensors where one tensor is actually locatedon another machine (is a PointerTensor). Call .get() on the PointerTensor or .send(anum) on the other tensor.

Tensor A: tensor([1, 1])
Tensor B: [PointerTensor | me:7104777789 -> anum:16499776846]

In [20]:
input = th.tensor([[1.,1],[0,1],[1,0],[0,0]],requires_grad=True)
target = th.tensor([[1.],[1],[0],[0]],requires_grad=True)
weights = th.tensor([[0.],[0]],requires_grad=True)

In [ ]:
nn.Linear(2,)

In [21]:
for i in range(10):
    pred = input.mm(weights)
    loss = ((pred-target)**2).sum()
    loss.backward()
    weights = weights - (weights.grad*0.1)
    print(loss)

tensor(2., grad_fn=<SumBackward0>)


TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'